# Fighter Calibration Model

## Configure your Fighters here: 

In [ ]:
# Define Fighter 1
fighter1_name = "Fighter #1"
fighter1_attack_range = [1,60]
fighter1_defense_range = [1,20]
fighter1_stamina_range = [1,45]
fighter1_speed_range = [1,25]

# Define Fighter 2
fighter2_name = "Fighter #2"
fighter2_attack_range = [1,20]
fighter2_defense_range = [1,60]
fighter2_stamina_range = [1,25]
fighter2_speed_range = [1,45]

number_of_fights = 1000
initial_health = 150

## Execute Fight(s)

In [ ]:
from Objects.Fighter import Fighter
from Objects.Fight import Fight

In [ ]:
for number in range(number_of_fights):
    f1 = Fighter(fighter1_name, fighter1_attack_range, fighter1_defense_range, fighter1_stamina_range, fighter1_speed_range, initial_health)
    f2 = Fighter(fighter2_name, fighter2_attack_range, fighter2_defense_range, fighter2_stamina_range, fighter2_speed_range, initial_health)
    
    f = Fight(number+1, f1, f2)

## Process Data

### Pick up and display sample of statistics file

In [ ]:
import pandas as pd

df = pd.read_csv('statistics.csv')
df.head(5)

### Summarize data and group

In [ ]:
df_fightround = df[['fight_number','fight_round']]
df_grouped = df_fightround.groupby('fight_number')
df_max = df_grouped.max().reset_index()

f1_winner_balance = df[df.fighter1_win == True].shape[0]/len(df)
f2_winner_balance = df[df.fighter2_win == True].shape[0]/len(df)

## Generate Chart

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

fig, axs = plt.subplots(2, 3, figsize=(13, 7))

# define axs[0,0] to [0,2]  (fighter stats)
fig.set_tight_layout(False)
axs[0,0].axis('off')
axs[0,1].axis('off')
axs[0,2].axis('off')
axs[0,0].text(0,0,f2.__str__(), va="center")
axs[0,0].text(0,0.1,f1.__str__(), va="center")

# define axs[1,0] (plot)

sns.set(style="darkgrid", font_scale=1.2)

chart = sns.histplot(
                    data=df_max,
                    x="fight_round",
                    ax=axs[1,0], 
                    discrete=True
                    )

chart.set(
            xticks=[1,2,3,4,5,6,7,8,9,10],
            xlabel="Number of Rounds",
            ylabel="Count of Fights",
            title="Distribution of Rounds",
            xlim=(0,11)
        )

# define axs[1,1] (table)

plt.rcParams["figure.autolayout"] = True
v = pd.DataFrame(df_max['fight_round'].value_counts()).sort_index()
axs[1,1].axis('off')
table = axs[1,1].table(cellText=v.values, rowLabels=v.index, bbox=[0,0,1,1], colLabels=["Count"])

# define axs[1,2] (winner/loser balance)

axs[1,2].pie([f1_winner_balance, f2_winner_balance],
           labels = ['Fighter 1', 'Fighter 2'], 
           colors = ['#7e8cbe','#ff9999'], autopct='%.2f%%')

axs[1,2].set_title('Rounds won')
plt.show()

## Move Statistics to Archive

In [ ]:
import os

counter = 0

# find the latest file number in archives, increase by 1
# if no file is found yet, start counter at 1
filelist = os.listdir("./Archives")
if  filelist == []:
    counter = 1
else:
    filelist.sort()
    counter = int(os.path.basename(filelist[-1])[0]) + 1

# save chart and table as PDF
fig.savefig("Archives/%i.pdf" % counter)

# move statistics file to archive
os.rename("statistics.csv", "Archives/%i.csv" % counter)